In [1]:
# add autoreload and other ipython magic
import glob
import os

import numpy as np
import pandas as pd
from IPython import get_ipython

ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")
ipython.run_line_magic("matplotlib", "inline")
ipython.run_line_magic("config", 'InlineBackend.figure_format = "retina"')

In [ ]:
# Replace with your folder path
folder = os.getcwd()

# List all matching CSV files
csv_files = glob.glob(os.path.join(folder, "results_*.csv"))

# Read and concatenate all into a single DataFrame
df_all = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

# Make the data longer for analysis
df_long = (
    df_all.melt(id_vars="seed", var_name="variable", value_name="value")
    .assign(
        metric=lambda x: x["variable"].str.extract(r"^([a-zA-Z]+)"),
        method=lambda x: x["variable"].str.extract(r"_(.*)$").fillna(""),
    )
    .drop(columns=["variable"])
    .pivot(index=["seed", "method"], columns="metric", values="value")
    .reset_index()
    .drop(columns=["seed"])
)

In [ ]:
def mean_ci(x, digits: int = 3) -> str:
    x = np.asarray(x, dtype=np.float64)
    n = x.size
    if n == 0:
        return "nan ± nan"
    mean = np.mean(x)
    se = np.std(x, ddof=1) / np.sqrt(n)
    ci_half_width = 1.96 * se
    return f"{round(mean, digits)} ± {round(ci_half_width, digits)}"


summary = df_long.groupby("method").agg(mean_ci)
summary = summary[["mse", "loglik", "mmd", "fid"]]
summary.columns.name = None
summary.index.name = None
summary

,mse,loglik,mmd,fid
initial,0.054 ± 0.001,-17.179 ± 1.131,0.514 ± 0.027,6.259 ± 0.099
refit,0.052 ± 0.001,-8.098 ± 1.199,0.492 ± 0.03,6.62 ± 0.197
